In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---
# Mouse
## Quake_Smart-seq2_Diaphragm

In [ ]:
quake_smart_seq2_diaphragm = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_Smart-seq2_Diaphragm/data.h5")
utils.peek(quake_smart_seq2_diaphragm, "build/diaphragm/Quake_Smart-seq2_Diaphragm")
quake_smart_seq2_diaphragm.obs.head()

In [ ]:
quake_smart_seq2_diaphragm.obs["cluster"] = "cluster_" + quake_smart_seq2_diaphragm.obs["cluster"].astype(int).astype(str)
quake_smart_seq2_diaphragm.obs.dtypes

In [ ]:
#opt_model_kwargs = dict(batch_effect="donor")
quake_smart_seq2_diaphragm_model = cb.directi.fit_DIRECTi(
    quake_smart_seq2_diaphragm, quake_smart_seq2_diaphragm.uns["seurat_genes"],
    **fixed_model_kwargs #**opt_model_kwargs
)
quake_smart_seq2_diaphragm.latent = quake_smart_seq2_diaphragm_model.inference(quake_smart_seq2_diaphragm)

In [ ]:
ax = quake_smart_seq2_diaphragm.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/diaphragm/Quake_Smart-seq2_Diaphragm/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_diaphragm.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/diaphragm/Quake_Smart-seq2_Diaphragm/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_diaphragm.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/diaphragm/Quake_Smart-seq2_Diaphragm/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_diaphragm.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/diaphragm/Quake_Smart-seq2_Diaphragm/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_diaphragm.visualize_latent("gender", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/diaphragm/Quake_Smart-seq2_Diaphragm/gender.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
quake_smart_seq2_diaphragm.write_dataset("build/diaphragm/Quake_Smart-seq2_Diaphragm/Quake_Smart-seq2_Diaphragm.h5")

In [ ]:
%%capture capio
quake_smart_seq2_diaphragm_models = [quake_smart_seq2_diaphragm_model]
opt_model_kwargs = dict(batch_effect="donor")
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    quake_smart_seq2_diaphragm_models.append(cb.directi.fit_DIRECTi(
        quake_smart_seq2_diaphragm, quake_smart_seq2_diaphragm.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
quake_smart_seq2_diaphragm_blast = cb.blast.BLAST(
    quake_smart_seq2_diaphragm_models, quake_smart_seq2_diaphragm, 
)
quake_smart_seq2_diaphragm_blast.save("build/diaphragm/Quake_Smart-seq2_Diaphragm")

In [ ]:
with open("build/diaphragm/Quake_Smart-seq2_Diaphragm/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/diaphragm/Quake_Smart-seq2_Diaphragm/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(quake_smart_seq2_diaphragm_blast, "build/diaphragm/Quake_Smart-seq2_Diaphragm")

In [ ]:
%%writefile build/diaphragm/Quake_Smart-seq2_Diaphragm/predictable.txt
cell_ontology_class
cell_type1
cluster